In [1]:
# Native
import sys
from itertools import combinations, chain

# External general
from tqdm import tqdm
import pandas as pd

# Proteomics imports
from unimod_mapper import UnimodMapper
from pyteomics.mass.mass import std_aa_mass

In [2]:
# Get list of all unimod modifications
unimod = UnimodMapper()
modifications = unimod.data_list

# Define type of mods to include
types_mod_include = set((
    "O-linked glycosylation",
    "Post-translational",
    "Chemical derivative",
    "Isotopic label",
    #"AA substitution",
    "Other",
    "Pre-translational",
    "NoMod",
    "Multiple"
))

2022-05-25 14:47:50.207 | DEBUG    | unimod_mapper.unimod_mapper:_parseXML:496 - No usermod.xml file found. Expected at C:\Users\robbin\anaconda3\envs\py39\lib\site-packages\unimod_mapper\usermod.xml
2022-05-25 14:47:50.209 | DEBUG    | unimod_mapper.unimod_mapper:_parseXML:439 - Parsing mods file (C:\Users\robbin\anaconda3\envs\py39\lib\site-packages\unimod_mapper\unimod.xml)


In [3]:
# Create a dictionary that contains a mapping between AA and all modifications

# aa_to_modifications[{AA}] = [ [unimodID,unimodName,monoisotopicmass,modificationType] , ... ]
aa_to_modifications = {}

for mod in modifications:
    for loc in mod["specificity"]:
        # loc[0] == AA and loc[1] == type of modification (e.g., chemical derivative)
        if loc[0] in aa_to_modifications.keys():
            aa_to_modifications[loc[0]].append([mod["unimodID"],mod["unimodname"],mod["mono_mass"],loc[1]])
        else:
            aa_to_modifications[loc[0]] = [[mod["unimodID"],mod["unimodname"],mod["mono_mass"],loc[1]]]

In [4]:
# List containing all the fragment ion and its monoisotopic mass
# [ [aminoacid+modification, monoisotopicMass] , ... ]
list_collision_possibility = []

for aa in std_aa_mass.keys():
    # O and J are not in unimod ...
    try:
        aa_mods = aa_to_modifications[aa]            
    except KeyError:
        print(f"Unable to analyze amino acid: {aa}, no modifications found in unimod")
        continue
    
    for modification in aa_mods:
        # Filter on modification type
        if modification[3] not in types_mod_include:
            continue
        # Filter on modifications that remove the actual amino acid + some extra
        if std_aa_mass[aa]+modification[2] < 0.0:
            continue
            
        # Write non-modified amino acids differently        
        if modification[3] == "NoMod":
            list_collision_possibility.append([f"{aa}",std_aa_mass[aa]+modification[2]])
        else:
            list_collision_possibility.append([f"{aa}[UNIMOD:{modification[0]}]",std_aa_mass[aa]+modification[2]])

Unable to analyze amino acid: J, no modifications found in unimod
Unable to analyze amino acid: O, no modifications found in unimod


In [5]:
print(f"Length of fragment ion list that is potentially colliding (single AA + mod): {len(list_collision_possibility)}")

Length of fragment ion list that is potentially colliding (single AA + mod): 1720


In [ ]:
# Repeat for diamino acids, so combinations of two amino acids and their modifications
# TODO more general code where the number of amino acids is simply defined
for aa1, aa2 in combinations(std_aa_mass.keys(), 2):
    try:
        aa_mods1 = aa_to_modifications[aa1]
        aa_mods1.append(["","",0,"NoMod"])
    except KeyError:
        print(f"Unable to analyze amino acid: {aa1}, no modifications found in unimod")
        continue
    try:
        aa_mods2 = aa_to_modifications[aa2]
        aa_mods2.append(["","",0,"NoMod"])
    except KeyError:
        print(f"Unable to analyze amino acid: {aa2}, no modifications found in unimod")
        continue
        
    for modification1 in aa_mods1:
        for modification2 in aa_mods2:
            if modification1[3] not in types_mod_include:
                continue
            if std_aa_mass[aa1]+modification1[2] < 0.0:
                continue
            
            if modification2[3] not in types_mod_include:
                continue
            if std_aa_mass[aa2]+modification2[2] < 0.0:
                continue

            if modification1[3] == "NoMod" and modification2[3] == "NoMod":
                list_collision_possibility.append([f"{aa1}{aa2}",std_aa_mass[aa]+modification[2]])
            elif modification1[3] == "NoMod":
                list_collision_possibility.append([f"{aa1}{aa2}[UNIMOD:{modification2[0]}]",
                                                   std_aa_mass[aa1]+modification1[2]+std_aa_mass[aa2]+modification2[2]])
            elif modification2[3] == "NoMod":
                list_collision_possibility.append([f"{aa1}[UNIMOD:{modification1[0]}]{aa2}",
                                                   std_aa_mass[aa1]+modification1[2]+std_aa_mass[aa2]+modification2[2]])
            else:
                list_collision_possibility.append([f"{aa1}[UNIMOD:{modification1[0]}]{aa2}[UNIMOD:{modification2[0]}]",
                                                   std_aa_mass[aa1]+modification1[2]+std_aa_mass[aa2]+modification2[2]])


In [ ]:
print(f"Length of fragment ion list that is potentially colliding (single AA + diamino + mod): {len(list_collision_possibility)}")

In [6]:
# Presort the list of potential fragment ions that could collide (makes it slightly faster)
list_collision_possibility = sorted(list_collision_possibility, key=lambda x:x[1])

In [7]:
# Define tolerance for colliding
tolerance_pos = 0.05
tolerance_neg = 0.05

dict_collission = {}

# Iterate over sorted list of all fragment ions
for idx,fragment in enumerate(tqdm(list_collision_possibility)):
    # Go up in the list of fragment ion masses until it goes over the tolerance value
    for diff_candidate in list_collision_possibility[idx+1:]:
        if diff_candidate[0] == fragment[0]:
            continue
        if diff_candidate[1]-fragment[1] > tolerance_pos:
            break
            
        dict_collission[f"{fragment[0]}->{diff_candidate[0]}"] = list(chain(*[fragment,diff_candidate]))
    
    # Go down in the list of fragment ion masses until it goes over the tolerance value
    for diff_candidate in list_collision_possibility[:idx+1][::-1]:
        if diff_candidate[0] == fragment[0]:
            continue
        if fragment[1]-diff_candidate[1] > tolerance_neg:
            break
            
        dict_collission[f"{fragment[0]}->{diff_candidate[0]}"] = list(chain(*[fragment,diff_candidate]))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1720/1720 [00:00<00:00, 102055.52it/s]


In [8]:
df_collision = pd.DataFrame(dict_collission).T
df_collision.columns = ["Fragment_selected","Monoisotopic_mass_fragment","Fragment_collision","Monoisotopic_mass_collision"]

In [9]:
df_collision.describe()

,Fragment_selected,Monoisotopic_mass_fragment,Fragment_collision,Monoisotopic_mass_collision
count,2004,2004.000000,2004,2004.000000
unique,900,841.000000,900,841.000000
top,S[UNIMOD:58],144.053494,K[UNIMOD:381],195.029661
freq,7,15.000000,7,15.000000


In [11]:
df_collision.to_csv("single_aa_modifications_collision.csv")